# Feature Engineering

### En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

### En este notebook se seleccionan, transforman o crean nuevas variables a partir de los datasets obtenidos en el ETL; con el fin de preparar los datos de entrada  hasta que sean los adecuados para el modelo de Machine Learning que luego se vaya a utilizar

## Importando Librerías

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp 
import ast
import numpy as np 


In [3]:
from textblob import TextBlob

### Se extraen los archivos parquet generados en el ETL para trabajar con el feature

 El archivo `steam_games` contiene la información sobre los juegos de la plataforma Steam :
   * **publisher**: es la empresa publicadora del contenido.
   * **genres**: es el género del item, es decir, del juego. Esta formado por una lista de uno o mas géneros por registro.
   * **app_name**: es el nombre del item, es decir, del juego.
   * **title**: es el título del item.
   * **url**: es la url del juego.
   * **release_date**: es la fecha de lanzamiento del item en formato 2018-01-04.
   * **tags**: es la etiqueta del contenido. Esta formado por una lista de uno o mas etiquetas por registro.
   * **reviews_url**: es la url donde se encuentra el review de ese juego.
   * **specs**: son especificaciones de cada item. Es una lista con uno o mas string con las especificaciones.
   * **price**: es el precio del item.
   * **early_access**: indica el acceso temprano con un True/False.
   * **id**: es el identificador único del contenido.
   * **developer**: es el desarrollador del contenido.
   * **año_estreno**: es el año de lanzamiento del item.

In [4]:
archivo="data/steam_games.parquet"
steam_games= pd.read_parquet(archivo)

El archivo `user_review` contiene información relacionada con las reviews que realizaron los usuarios de los juegos:
* **user_id**: es un identificador único para el usuario.
* **user_url**: es la url del perfil del usuario en streamcommunity.
* **reviews**: contiene una lista de diccionarios. Para cada usuario se tiene uno o mas diccionario con el review. Cada diccionario contiene:
    *    **funny**: indica si alguien puso emoticón de gracioso al review.
    *   **posted**: es la fecha de posteo del review en formato Posted April 21, 2011.
    *   **last_edited**: es la fecha de la última edición.
    *   **item_id**: es el identificador único del item, es decir, del juego.
    *   **helpful**: es la estadística donde otros usuarios indican si fue útil la información.
    *   **recommend**: es un booleano que indica si el usuario recomienda o no el juego.
    *   **review**: es una sentencia string con los comentarios sobre el juego.

In [5]:
archivo="data/user_review.parquet"
user_reviews= pd.read_parquet(archivo)

In [6]:
user_reviews 

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-11-05,None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-07-15,None,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-04-21,None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,None,2014-06-24,None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,None,2013-09-08,None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59156,Fuckfhaisjnsnsjakaka,http://steamcommunity.com/id/Fuckfhaisjnsnsjakaka,None,None,None,70,No ratings yet,True,a must have classic from steam definitely wort...
59157,3214213216,http://steamcommunity.com/id/3214213216,None,None,None,362890,No ratings yet,True,this game is a perfect remake of the original ...
59158,ChrisCoroner,http://steamcommunity.com/id/ChrisCoroner,1 person found this review funny,None,None,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59159,CaptainAmericaCw,http://steamcommunity.com/id/CaptainAmericaCw,None,None,None,730,No ratings yet,True,:D


El archivo `user_items` contiene información con respecto al consumo de juegos de la plataforma por parte de los usuarios:
* **user_id**: contiene un identificador único del usuario.
* **items_count**: contiene un número entero que indica la cantidad de juegos que ha consumido el usuario.
* **steam_id**: es un número único para la plataforma.
* **user_url**: es la url del perfil del usuario
* **items**: contiene una lista de uno o mas diccionarios de los items que consume cada usuario. Cada diccionario tiene las siguientes claves:
    * **item_id**: es el identificados del item, es decir, del juego.
    * **item_name**: es el nombre del contenido que consume, es decir, del juego.
    * **playtime_forever**: es el tiempo acumulado que un usuario jugó a un juego.

In [6]:
archivo="data/user_items.parquet"
user_items= pd.read_parquet(archivo)

## Análisis de sentimientos

Se utiliza el dataframe `user_reviews` , para crear la columna "sentiment_analisis" y se aplica el análisis de sentimientos con la biblioteca NLP (Natural Lenguaje Toolkit) ó TextBlob, que implica determinar polaridad (positiva, negativa o neutra) de un texto como comentario o reseña.
La escala a utilizar es:
* 0 Malo
* 1 Neutral
* 2 Positivo

Se aplica sobre la columna "review", la que luego de creada la columna "sentiment_analisis"; será eliminada.

In [7]:

#La funcion analizar sentimiento se utiliza para que dado un texto en este caso una review, devuleva un valor que represente ese sentimiento de acuerdo a los parametros cargados en polaridad siendo mayor a 0.1 Positivo, menor que -0,1 Negativo y en caso contrario Neutral 
""" Parametros:
    -review (str): Eltexto de la review a analizar
    
    Return:
    -numero (int): Valor que se le cargo en la funcion de polaridad
                - 0 Negativo o Malo
                - 1 Neutral
                - 2 Positivo
"""
def analizar_sentimiento(review):
    if review is None:
        return 1 #Neutral
    analisis = TextBlob(review)
    polaridad = analisis.sentiment.polarity
    if polaridad > 0.1:
        return 2 #'Positivo'
    elif polaridad < -0.1:
        return 0 #'Negativo'
    else:
        return 1 #'Neutral'

In [8]:
user_reviews ["sentiment_analisis"]= user_reviews["review"].astype(str).apply(analizar_sentimiento)
user_reviews.head(5)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,sentiment_analisis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-11-05,None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-07-15,None,22200,No ratings yet,True,It's unique and worth a playthrough.,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-04-21,None,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1
3,js41637,http://steamcommunity.com/id/js41637,None,2014-06-24,None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
4,js41637,http://steamcommunity.com/id/js41637,None,2013-09-08,None,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1


In [9]:
#Se busca una reseña y se controla que el analisis este correcto
user_reviews["review"][9]

'Random drops and random quests, with stat points.  Animation style reminiscent of the era before the Voodoo card.'

In [10]:
user_reviews["sentiment_analisis"][9]

0

In [11]:
user_reviews["review"][1183]

'ONE OF THE BEST ♥♥♥♥ING GAMES I HAVE EVER PLAYED'

In [12]:
user_reviews["sentiment_analisis"][1183]

2

Una vez controladas la salidas de la función `analizar_sentimiento`, se elimina la columna "review" del dataframe `user_reviews`

In [13]:
user_reviews = user_reviews.drop(columns=["review"])
user_reviews.columns

Index(['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id',
       'helpful', 'recommend', 'sentiment_analisis'],
      dtype='object')

# Desarrollo de API

### Crear las funciones necesarias para los endpoints que consumiran las API

* **def userdata**( User_id : str ): Debe devolver `cantidad de dinero` gastado por el usuario, el `porcentaje de recomendación` en base a reviews.recommend y `cantidad de items`.

* **def countreviews**( YYYY-MM-DD y YYYY-MM-DD : str ): `Cantidad de usuarios` que realizaron reviews entre las fechas dadas y, el `porcentaje de recomendación` de los mismos en base a reviews.recommend.

* **def genre**( género : str ): Devuelve el `puesto` en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

* **def userforgenre**( género : str ): `Top 5` de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

* **def developer**( desarrollador : str ): `Cantidad de items` y `porcentaje de contenido Free` por año según empresa desarrolladora. Ejemplo de salida:

    | Activision ||
    |----------|----------|
    | Año  | Contenido Free  |
    | 2023   | 27% |
    | 2022    | 25%   |
    | xxxx    | xx%   |



* **def sentiment_analysis**( año : int ): Según el `año de lanzamiento`, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

                    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

### Función def **userdata(user_id:str)**

Debe devolver `cantidad de dinero` gastado por el usuario, el `porcentaje de recomendación` en base a reviews.recommend y `cantidad de items`.
Para ello se debe lograr un nuevo dataframe con los datos necesarios a recorrer por la función
- Se procede a analizar los tipos de datos de los dataframes
- Extraer las columnas necesarias
- Unirlas 

In [14]:
steam_games.dtypes

genres             object
url                object
release_date       object
tags               object
reviews_url        object
discount_price    float64
specs              object
price             float64
early_access         bool
id                  int64
metascore         float64
Año_estreno        object
publisher          object
app_name           object
title              object
developer          object
dtype: object

In [15]:
user_reviews.dtypes

user_id               object
user_url              object
funny                 object
posted                object
last_edited           object
item_id                int64
helpful               object
recommend               bool
sentiment_analisis     int64
dtype: object

In [36]:
#calculo cantidad de recomendaciones del dataframe
r=len(user_reviews["recommend"])
r

59161

In [37]:
#calculo cantidad de usuarios
total=len(user_reviews["user_id"].unique())
total

25467

In [16]:
user_items.dtypes

item_id              int64
item_name           object
playtime_forever     int64
steam_id             int64
items_count          int64
user_id             object
user_url            object
dtype: object

In [19]:
#Utilizo las columnas "user_id","item_id","items_count" del dataframe user_items para luego utilizarlas en las funciones
cant_items= user_items[["user_id","item_id","items_count"]]
cant_items

,user_id,item_id,items_count
0,76561197970982479,10,277
1,76561197970982479,20,277
2,76561197970982479,30,277
3,76561197970982479,40,277
4,76561197970982479,50,277
...,...,...,...
5094100,76561198329548331,346330,7
5094101,76561198329548331,373330,7
5094102,76561198329548331,388490,7
5094103,76561198329548331,521570,7


In [21]:
#Utilizo la columna "recommend" y su "user_id" del dataframe user_reviews
recommend= user_reviews[["user_id", "recommend"]]
recommend

,user_id,recommend
0,76561197970982479,True
1,76561197970982479,True
2,76561197970982479,True
3,js41637,True
4,js41637,True
...,...,...
59156,Fuckfhaisjnsnsjakaka,True
59157,3214213216,True
59158,ChrisCoroner,True
59159,CaptainAmericaCw,True


In [25]:
steam_games["price"].isnull().sum()

0

In [29]:
# Y utilizo la columna "price" del dataframe steam_games , luego cambio el nombre de id por "item_id" a fin de poder realizar
# el merge
cant_dinero = steam_games[["price", "id"]] 
cant_dinero = cant_dinero.drop_duplicates("id",keep="first")
cant_dinero = cant_dinero.rename(columns={"id":"item_id"})
cant_dinero


,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71535,1.99,745400
71539,1.99,773640
71543,4.99,733530
71546,1.99,610660


Una vez separadas las columnas que se necesitan de cada dataframe original , se comienzan a unir a traves de el método merge, que vincula las columnas por medio de una columna de unión en común 

In [30]:
cant_items= cant_items.merge(cant_dinero, on="item_id", how="left")
cant_items

,user_id,item_id,items_count,price
0,76561197970982479,10,277,9.99
1,76561197970982479,20,277,4.99
2,76561197970982479,30,277,4.99
3,76561197970982479,40,277,4.99
4,76561197970982479,50,277,4.99
...,...,...,...,...
5094100,76561198329548331,346330,7,0.00
5094101,76561198329548331,373330,7,NaN
5094102,76561198329548331,388490,7,0.00
5094103,76561198329548331,521570,7,0.00


Los datos nulos en la columna precio se reemplazan por 0.0

In [31]:
cant_items ["price"].isnull().sum()

937657

In [32]:
# se define una función para poder reemplazar los datos string de la columna "price" por flotantes 0.0

def cambio_a_float(valor):

    if pd.isna(valor): #esta parte de la función es para el caso de que no exista ningun string o float en el campo
        return 0.0
    try:
        flotante = float(valor) #si el valor es float lo conservo
        return flotante
    except (ValueError, TypeError): #si el valor es diferente retorno 0.0
        return 0.0
    
#Aplico esta función a la columna precio
cant_items["price"] = cant_items["price"].apply(cambio_a_float)

In [33]:
cant_items ["price"].isnull().sum()

0

In [40]:
#La siguiente función retorna información sobre el usuario que se le pasa como argumento
"""
Parametro: 
        user_id(str) : ID del Usuario a consultar.
Retorna:
        user (dict): Información de un usuario ,
        -cantidad de dinero gastado (int): Dinero gastado por usuario
        -Porcentaje de recomendación usuario (float): Reviews realizadas por el usuario con respecto a la cantidad de 
        reviews poe usuario
        -cantidad de items (int):cantidad de juegos consumidos por usuario 
"""
def userdata(user_id):
    #igualo el user_id al user_id del dataframe con los datos de los reviews 
    user= user_reviews[user_reviews["user_id"]== user_id]
    #sumo la columna price del dataframe cant_items par conocer el gasto por usuario
    gasto = cant_items[cant_items["user_id"]== user_id]["price"].sum()
    
    #cantidad de recomendaciones del usuario ingresado
    rec_user= recommend[recommend["user_id"]== user_id]["recommend"].sum()
    #cantidad de recomendaciones totales por usuario
    total_rec= len(user_reviews["user_id"].unique())
    porcentaje=(rec_user/total_rec)*100
    
    #cuento la cantidad de jueagos que utilizo el usuario 
    count= cant_items[cant_items["user_id"]== user_id]["items_count"].iloc[0]
    return{
        "Cantidad de dinero gastado": int (gasto),
        "Porcentaje de recomendación usuario": round(float(porcentaje), 3),
        "Cantidad de items": int(count)
    }

In [41]:
user_id= "EchoXSilence"
userdata(user_id)

{'Cantidad de dinero gastado': 189,
 'Porcentaje de recomendación usuario': 0.004,
 'Cantidad de items': 23}

### Función **def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ):**

`Cantidad de usuarios` que realizaron reviews entre las fechas dadas y, el `porcentaje de recomendación` de los mismos en base a reviews.recommend.

In [45]:
# La siguiente función retorna la cantidad de usuarios que realizaron reviews y el porcentaje de reviews de estos con respecto al total de usuarios en un ranfo de fechas
"""
Parametros:
        -f_inicio(str/datetime): Fecha de inicio del rango a evaluar 
        -f_final(str/datetime): Fecha final del rango a evaluar 
Retorna
        -Cantidad de usuarios: con reseñas dentro de ese período de tiempo
        -Porcentaje de reviews entre fechas : del usuario con respecto al total en el período
"""
def countreviews(f_inicio,f_final):
    #convierte las fechas a objetos datetime en el caso de que no lo estén
    #f_inicio = pd.to_datetime(f_inicio)
    #f_final = pd.to_datetime (f_final)
    
    #crea el rango de fechas mediante el filtro
    rango_fechas=user_reviews[(user_reviews['posted']>=f_inicio)&(user_reviews['posted']<=f_final)]
    
    #calculo de la cantidad de usuarios que hicieron reviews entre esas fechas
    count_usu= rango_fechas["user_id"].nunique()
    
    #calculo el porcentaje de reviews en el mismo rango
    porcentaje_fechas=(rango_fechas["recommend"].sum() / len(rango_fechas))* 100
    
    return{
        "Cantidad de usuarios con reseñas" : count_usu ,
        "Porcentaje de reviews entre fechas": round(float(porcentaje_fechas), 3)
    }

In [52]:
countreviews('2011-11-25', '2011-12-18')

{'Cantidad de usuarios con reseñas': 35,
 'Porcentaje de reviews entre fechas': 97.436}

### Función **def genre**( género : str )

Devuelve el `puesto` en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

Para ello se debe tener un dataframe con algunas columnas de `steam_games` y de `user_items`

In [54]:
#Se extraen las columnas para generar un nuevo dataframe y correr la función
genre= steam_games[["genres","id"]]
#Cambiamos "id" a "item_id" para poder utilizar el merge
genre= genre.rename(columns={"id":"item_id"})
genre

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
71546,Indie,610660
71547,Racing,610660
71548,Simulation,610660
71549,Casual,658870


In [56]:
play_time= user_items[["item_id","playtime_forever"]]
play_time

,item_id,playtime_forever
0,10,6
1,20,0
2,30,7
3,40,0
4,50,0
...,...,...
5094100,346330,0
5094101,373330,0
5094102,388490,3
5094103,521570,4


In [57]:
genre= play_time.merge(genre, on="item_id")
genre

,item_id,playtime_forever,genres
0,10,6,Action
1,10,0,Action
2,10,0,Action
3,10,93,Action
4,10,108,Action
...,...,...,...
9877299,354280,164,Indie
9877300,354280,164,Simulation
9877301,433920,0,Adventure
9877302,433920,0,Indie


In [58]:
genre= genre.drop("item_id", axis=1)
genre

,playtime_forever,genres
0,6,Action
1,0,Action
2,0,Action
3,93,Action
4,108,Action
...,...,...
9877299,164,Indie
9877300,164,Simulation
9877301,0,Adventure
9877302,0,Indie


Se crea un dataframe con la columna "ranking" para utilizar en la función

In [59]:
#agrupo los generos por la columna "playtime_forever" y reseteo el indice
rank_genre= genre.groupby("genres")["playtime_forever"].sum().reset_index()
#ordeno los valores
rank_genre= rank_genre.sort_values(by="playtime_forever", ascending=False)
#crear la columna de ranking
rank_genre['ranking']= range (1,len(rank_genre)+1 )  #se empieza a contar desde uno 
rank_genre

,genres,playtime_forever,ranking
0,Action,3075100464,1
9,Indie,1475473529,2
12,RPG,1027927921,3
1,Adventure,898763639,4
14,Simulation,855263068,5
17,Strategy,650996822,6
8,Free to Play,603643302,7
10,Massively Multiplayer,441125566,8
4,Casual,249316634,9
6,Early Access,156692536,10


In [60]:
# La siguiente función retorna el ranking en que se ubica el genero que se le ingresa de acuerdo a "playtime_forever"
"""
Parametros:
        - genero (str): el genero de juegos Steam que se quiera conocer el tiempo jugado
Retorna:
        - orden: La ubicación dentro de ranking de acuerdo a la columna "playtime_forever"
""" 
def genre(genero):
    #filtro el dataframe "rank_genre" para quesu columna "genres" sea igual a el dato que se ingresa
    # a partir de esto se selecciona la columna "ranking" del conjunto resultante y se bloquea para obtener el valor
    orden= rank_genre[rank_genre["genres"]== genero]["ranking"].iloc[0]
    return {
        "El género": genero, 
        "se ubica en el raking de PlayTimeForever": orden
    }

In [61]:
genre("Simulation")


{'El género': 'Simulation', 'se ubica en el raking de PlayTimeForever': 5}

### Función **def userforgenre**( género : str ):

`Top 5` de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

Para ello se debe tener un dataframe con algunas columnas de `steam_games` y de `user_items`

In [162]:

genre= steam_games[["genres","id"]]
#Cambiamos "id" a "item_id" para poder utilizar el merge
genre= genre.rename(columns={"id":"item_id"})
genre

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
71546,Indie,610660
71547,Racing,610660
71548,Simulation,610660
71549,Casual,658870


In [163]:
play= user_items[["item_id","playtime_forever", "user_id","user_url"]]
play

,item_id,playtime_forever,user_id,user_url
0,10,6,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,7,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...
5094100,346330,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094101,373330,0,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094102,388490,3,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5094103,521570,4,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [164]:
play= play.merge(genre, on= "item_id")
play

,item_id,playtime_forever,user_id,user_url,genres
0,10,6,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Action
1,10,0,js41637,http://steamcommunity.com/id/js41637,Action
2,10,0,Riot-Punch,http://steamcommunity.com/id/Riot-Punch,Action
3,10,93,doctr,http://steamcommunity.com/id/doctr,Action
4,10,108,corrupted_soul,http://steamcommunity.com/id/corrupted_soul,Action
...,...,...,...,...,...
9877299,354280,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Indie
9877300,354280,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Simulation
9877301,433920,0,inven,http://steamcommunity.com/id/inven,Adventure
9877302,433920,0,inven,http://steamcommunity.com/id/inven,Indie


In [165]:
play= play.drop("item_id", axis= 1)
play

,playtime_forever,user_id,user_url,genres
0,6,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Action
1,0,js41637,http://steamcommunity.com/id/js41637,Action
2,0,Riot-Punch,http://steamcommunity.com/id/Riot-Punch,Action
3,93,doctr,http://steamcommunity.com/id/doctr,Action
4,108,corrupted_soul,http://steamcommunity.com/id/corrupted_soul,Action
...,...,...,...,...
9877299,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Indie
9877300,164,76561198107283457,http://steamcommunity.com/profiles/76561198107...,Simulation
9877301,0,inven,http://steamcommunity.com/id/inven,Adventure
9877302,0,inven,http://steamcommunity.com/id/inven,Indie


In [167]:
#agrupo el genero por usuario y horas jugadas
user_hours= play.groupby(["genres","user_id","user_url"])["playtime_forever"].sum().reset_index()
    #ordena los valores de mayor a menor
user_hours= user_hours.sort_values(by="playtime_forever", ascending=False)
user_hours

,genres,user_id,user_url,playtime_forever
343532,Indie,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,2402994
112560,Adventure,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,2191551
50657,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307
64497,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428
49457,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212
...,...,...,...,...
632464,Strategy,76561198093437165,http://steamcommunity.com/profiles/76561198093...,0
213485,Early Access,76561198086028360,http://steamcommunity.com/profiles/76561198086...,0
141196,Casual,76561198036817186,http://steamcommunity.com/profiles/76561198036...,0
213481,Early Access,76561198086008401,http://steamcommunity.com/profiles/76561198086...,0


In [170]:
# La siguiente función retorna el TOP 5 de usuarios junto a su información, con mayor horas de juego en el genero que se le indica
"""
Parametros:
        -genero (str): El genero de juego Steam del que se necesita conocer el TOP 5 de usuarios
Retorna:
        -top_5_users(list) : Lista ordenada por horas de juego "playtime_forever", en forma descendente, conteniendo los nombres de usuario y dirección url
"""
def userforgenre (genero):
    #se filtra el dataframe con la columna "genres" y se la iguala con el dato ingresado
    genre_data= user_hours[user_hours["genres"]== genero]

    #extrae los primeros 5 
    top_5_users= genre_data.head(5)
    return top_5_users

In [171]:
genero= "Action"
userforgenre(genero)

,genres,user_id,user_url,playtime_forever
50657,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307
64497,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580428
49457,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212
51237,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065742
44133,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193


### Función **def developer**( desarrollador : str )

`Cantidad de items` y `porcentaje de contenido Free` por año según empresa desarrolladora. Ejemplo de salida:

| Activision ||
|----------|----------|
| Año  | Contenido Free  |
| 2023   | 27% |
| 2022    | 25%   |
| xxxx    | xx%   |

Por lo que se busca es un dataframe que contenga para cada desarrollador de juegos Steam, el año de lanzamiento y precio de cada uno

In [76]:
#Se extraen columnas del dataframe "steam_games" 
devs=steam_games[["id","Año_estreno","developer", "price"]]
#Se renombran las columnas necesarias
devs= devs.rename(columns={"id":"item_id", "Año_estreno":"release_anio"})
devs

,item_id,release_anio,developer,price
0,761140,2018,Kotoshiro,4.99
1,761140,2018,Kotoshiro,4.99
2,761140,2018,Kotoshiro,4.99
3,761140,2018,Kotoshiro,4.99
4,761140,2018,Kotoshiro,4.99
...,...,...,...,...
71546,610660,2018,Laush Dmitriy Sergeevich,1.99
71547,610660,2018,Laush Dmitriy Sergeevich,1.99
71548,610660,2018,Laush Dmitriy Sergeevich,1.99
71549,658870,2017,"xropi,stev3ns",4.99


A simple vista se reconocen duplicados en los item_id, por lo que se eliminan

In [77]:
devs= devs.drop_duplicates()
devs

,item_id,release_anio,developer,price
0,761140,2018,Kotoshiro,4.99
5,643980,2018,Secret Level SRL,0.00
9,670290,2017,Poolians.com,0.00
14,767400,2017,彼岸领域,0.99
17,772540,2018,Trickjump Games Ltd,3.99
...,...,...,...,...
71535,745400,2018,Bidoniera Games,1.99
71539,773640,2018,"Nikita ""Ghost_RUS""",1.99
71543,733530,2018,Sacada,4.99
71546,610660,2018,Laush Dmitriy Sergeevich,1.99


In [93]:
#La siguiente función retorna la cantidad y porcentaje de juegos gratis por desarrollador y año
"""
Parametros:
        -desarrollador (str): El desarrollador del juego Steam que se ingresa 
Retorna: un dataframe
        -Año: año en que se da el estreno del juego 
        -Cantidad de items por año: cantidad de juegos publicados por el desarrollador en el año 
        -Porcentaje de juegos free: porcentaje de juegos gratis con respecto a los publicados en ese año
"""
def developer(desarrollador):
    
    #Se filtra el dataframe devs para igualarlo al dato que se ingresa
    data= devs[devs["developer"]== desarrollador]
    
    #Se agrupa por año para contar los items por año
    cantidad = data.groupby("release_anio")["item_id"].count()
    #Se agrupa por price para encontrar la cantidad free
    free_anio= data[data["price"]== 0.0].groupby("release_anio")["item_id"].count()
    porcentaje_gratis= (free_anio/cantidad*100).fillna(0).astype(int)
    
    #se crea una salida como dataframe
    tabla= pd.DataFrame({
        "Año": cantidad.index, #indice
        "Cantidad de items por año" : cantidad.values, #valor
        "Porcentaje de juegos free" : porcentaje_gratis.values #valor
    })
    return tabla

In [96]:
desarrollador = "Poolians.com"
developer(desarrollador)

,Año,Cantidad de items por año,Porcentaje de juegos free
0,2017,1,100


### Función **def sentiment_analysis**( año : int )

Según el `año de lanzamiento`, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

                    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [145]:
#Se extraen columnas del dataframe "steam_games" 
anio_estreno=steam_games[["id","Año_estreno"]]
#Se renombran las columnas necesarias
anio_estreno= anio_estreno.rename(columns={"id":"item_id", "Año_estreno":"release_anio"})
anio_estreno

,item_id,release_anio
0,761140,2018
1,761140,2018
2,761140,2018
3,761140,2018
4,761140,2018
...,...,...
71546,610660,2018
71547,610660,2018
71548,610660,2018
71549,658870,2017


In [146]:
#se eliminan duplicados para realizar el merge 
anio_estreno= anio_estreno.drop_duplicates()
anio_estreno

,item_id,release_anio
0,761140,2018
5,643980,2018
9,670290,2017
14,767400,2017
17,772540,2018
...,...,...
71535,745400,2018
71539,773640,2018
71543,733530,2018
71546,610660,2018


In [149]:
anio_estreno ["release_anio"]= anio_estreno["release_anio"].astype (int, errors="ignore")
anio_estreno.dtypes

C:\Users\JIME\AppData\Local\Temp\ipykernel_9176\5810914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anio_estreno ["release_anio"]= anio_estreno["release_anio"].astype (int, errors="ignore")


item_id          int64
release_anio    object
dtype: object

In [150]:
anio_estreno

,item_id,release_anio
0,761140,2018
5,643980,2018
9,670290,2017
14,767400,2017
17,772540,2018
...,...,...
71535,745400,2018
71539,773640,2018
71543,733530,2018
71546,610660,2018


In [151]:
#se le anexa el año de estreno al dataframe "user_reviews"
sentimiento_analysis=user_reviews.merge(anio_estreno, on="item_id")
sentimiento_analysis

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analisis,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,2011-11-05,None,1250,No ratings yet,True,2,2009
1,death-hunter,http://steamcommunity.com/id/death-hunter,None,2015-03-30,None,1250,No ratings yet,True,2,2009
2,DJKamBer,http://steamcommunity.com/id/DJKamBer,None,2013-07-12,None,1250,No ratings yet,True,2,2009
3,diego9031,http://steamcommunity.com/id/diego9031,None,2015-08-13,None,1250,No ratings yet,True,1,2009
4,76561198081962345,http://steamcommunity.com/profiles/76561198081...,None,2014-04-05,None,1250,No ratings yet,True,1,2009
...,...,...,...,...,...,...,...,...,...,...
50325,NikitaOsworth1991,http://steamcommunity.com/id/NikitaOsworth1991,None,2015-10-29,None,307130,1 of 5 people (20%) found this review helpful,True,1,2014
50326,pigeonie,http://steamcommunity.com/id/pigeonie,None,None,None,209120,3 of 6 people (50%) found this review helpful,True,1,2012
50327,wayfeng,http://steamcommunity.com/id/wayfeng,1 person found this review funny,None,Last edited August 13.,220090,3 of 6 people (50%) found this review helpful,True,0,2013
50328,wayfeng,http://steamcommunity.com/id/wayfeng,None,None,None,262850,0 of 1 people (0%) found this review helpful,True,0,2014


In [157]:
def sentiment_analysis(anio):
    #Se filtran las reviews por año y las igualo al año que se ingresa en la consulta transformandolo en string 
    reviews_por_anio= sentimiento_analysis[sentimiento_analysis["release_anio"]== str(anio)]
    
    #Se inicia una lista vacia por cada sentimiento para ir contandolos 
    Negativos = 0
    Neutral = 0
    Positivos = 0
    
    #Se itera sobre las filas de reviews_por_anio y se distibuyen los datos segun la columna "sentiment_analysis"
    for i in reviews_por_anio["sentiment_analisis"]:
        if i == 0:
            Negativos += 1
        elif i == 1:
            Neutral += 1 
        elif i == 2:
            Positivos += 1

    count_sentiment ={"Negative": Negativos , "Neutral" : Neutral, "Positive": Positivos}
    
    return count_sentiment

In [172]:
anio = 2015
sentiment_analysis(anio)

{'Negative': 989, 'Neutral': 2873, 'Positive': 2748}

## Creación de archivos

Se crean los archivos de dataframes en formato parquet para cada una funciones creadas

In [174]:
recommend.to_parquet("data/recommend.parquet")
cant_items.to_parquet("data/cant_items.parquet")
rank_genre.to_parquet("data/rank_genre.parquet")
user_hours.to_parquet("data/user_hours.parquet")
devs.to_parquet("data/devs.parquet")
sentimiento_analysis.to_parquet("data/sentimiento_analysis.parquet")